In [1]:
#importing modules needed for analysis
import matplotlib.pyplot as plt
import matplotlib
import pathlib
import numpy as np
import mne

In [2]:
#using a specific backend of the matplotlib module for images
matplotlib.use('Qt5Agg') 

In [3]:
#importing the dataframe from the sample data set provided by mne module for eeg analysis
sample_data_dir = mne.datasets.sample.data_path()  
sample_data_dir = pathlib.Path(sample_data_dir)
sample_data_dir

WindowsPath('C:/Users/Dell/mne_data/MNE-sample-data')

In [4]:
#extracting the raw data from the dataset imported
raw_path = sample_data_dir / 'MEG' / 'sample' / 'sample_audvis_raw.fif'
raw = mne.io.read_raw(raw_path)
raw

Opening raw data file C:\Users\Dell\mne_data\MNE-sample-data\MEG\sample\sample_audvis_raw.fif...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
    Range : 25800 ... 192599 =     42.956 ...   320.670 secs
Ready.


<Raw | sample_audvis_raw.fif, 376 x 166800 (277.7 s), ~3.2 MB, data not loaded>

In [5]:
#Visualizing the sensor locations as topoplot and 3D view
raw.plot_sensors(ch_type='eeg')
raw.plot_sensors(kind='3d', ch_type='eeg') 

<Figure size 640x640 with 1 Axes>

In [6]:
#selecting only eeg channel data from medical dataset
raw_eeg = raw.copy().pick_types(meg=False, eeg=True, eog=True, exclude=[])
len(raw_eeg.ch_names)   

Removing projector <Projection | PCA-v1, active : False, n_channels : 102>
Removing projector <Projection | PCA-v2, active : False, n_channels : 102>
Removing projector <Projection | PCA-v3, active : False, n_channels : 102>


61

In [7]:
#calculating sampling rate=100
raw_eeg_cropped = raw_eeg.copy().crop(tmax=100)
raw_eeg_cropped.times[-1]

99.99916914588277

In [8]:
#cropping and filtering the raw data using an FIR Band-pass Filter
raw_eeg_cropped.load_data()   
raw_eeg_cropped_filtered = raw_eeg_cropped.copy().filter(l_freq=0.1, h_freq=40)

Reading 0 ... 60061  =      0.000 ...    99.999 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 19821 samples (33.001 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    0.7s finished


In [9]:
#plotting the raw data
raw_eeg_cropped.plot()
#plotting filtered data 
raw_eeg_cropped_filtered.plot()

Using matplotlib as 2D backend.


<MNEBrowseFigure size 800x749 with 4 Axes>

Channels marked as bad:
['EEG 053']
Channels marked as bad:
['EEG 053']


In [ ]:
#plotting Patient Specific Directions
import matplotlib.pyplot as plt

fig, ax = plt.subplots(2)
#subplot 1 of PSD of raw data
raw_eeg_cropped.plot_psd(ax=ax[0], show=False)
#subplot 2 of PSD of filtered data
raw_eeg_cropped_filtered.plot_psd(ax=ax[1], show=False)

ax[0].set_title('PSD before filtering')
ax[0].set_xlabel('Frequency (Hz)')
ax[1].set_title('PSD after filtering')
ax[1].set_xlabel('Frequency (Hz)')
fig.set_tight_layout(True)
plt.show()

#graph can be clicked and dragged to get average power in that frequency range
#vertical dotted line indicates HPF cut-off frequency

NOTE: plot_psd() is a legacy function. New code should use .compute_psd().plot().
Effective window size : 0.426 (s)
NOTE: plot_psd() is a legacy function. New code should use .compute_psd().plot().
Effective window size : 0.426 (s)


C:\Users\Dell\AppData\Local\Temp\ipykernel_6784\3963371156.py:15: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.show()
C:\Users\Dell\AppData\Roaming\Python\Python311\site-packages\mne\viz\evoked.py:77: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  event.canvas.draw()
C:\Users\Dell\AppData\Roaming\Python\Python311\site-packages\mne\viz\evoked.py:84: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  event.canvas.draw()
C:\Users\Dell\AppData\Roaming\Python\Python311\site-packages\mne\viz\evoked.py:111: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  evoked_fig.canvas.draw()
C:\Users\Dell\AppData\Roaming\Python\Python311\site-packages\mne\viz\evoked.py:156: UserWarning: This figure includes Axes that are not compatible w

In [ ]:
#saving filtered data
raw_eeg_cropped_filtered.save(pathlib.Path('out_data') / 'eeg_cropped_filt_raw.fif', 
                              overwrite=True)